In [1]:
import pandas as pd

In [2]:
df_orders = pd.read_json(
    "../data/order_data_dtypes.json",
    orient='table',
)

In [3]:
df_orders.head()

,order_datetime,customer_id,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
0,2015-06-20 19:00:00,000097eabfd9,1,0,0.0,0.000,11.46960,5803498,20326,1779,30231,4356
1,2016-01-29 20:00:00,0000e2c6d9be,1,0,0.0,0.000,9.55800,239303498,76547,1619,30359,4356
2,2017-02-26 19:00:00,000133bb597f,1,0,0.0,0.493,5.93658,206463498,33833,1619,30359,4324
3,2017-02-05 17:00:00,00018269939b,1,0,0.0,0.493,9.82350,36613498,99315,1619,30359,4356
4,2015-08-04 19:00:00,0001a00468a6,1,0,0.0,0.493,5.15070,225853498,16456,1619,29463,4356


In [4]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 786547 entries, 0 to 786546
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_datetime       786547 non-null  datetime64[ns]
 1   customer_id          786547 non-null  object        
 2   customer_order_rank  786547 non-null  int64         
 3   is_failed            786547 non-null  int64         
 4   voucher_amount       786547 non-null  float64       
 5   delivery_fee         786547 non-null  float64       
 6   amount_paid          786547 non-null  float64       
 7   restaurant_id        786547 non-null  category      
 8   city_id              786547 non-null  category      
 9   payment_id           786547 non-null  category      
 10  platform_id          786547 non-null  category      
 11  transmission_id      786547 non-null  category      
dtypes: category(5), datetime64[ns](1), float64(3), int64(2), object(1)
memor

In [5]:
# Customer order rank
max_customer_order_rank = df_orders.groupby(
    'customer_id'
)['customer_order_rank'].max()

In [6]:
max_customer_order_rank

customer_id
000097eabfd9    1
0000e2c6d9be    1
000133bb597f    1
00018269939b    1
0001a00468a6    1
               ..
fffd696eaedd    1
fffe9d5a8d41    1
ffff347c3cfa    2
ffff4519b52d    1
ffffccbfc8a4    1
Name: customer_order_rank, Length: 245453, dtype: int64

In [7]:
df_orders['is_holiday'] = (
    (
        (df_orders['order_datetime'].dt.month == 1) & (df_orders['order_datetime'].dt.day == 1)
    ) | (
        (df_orders['order_datetime'].dt.month == 12) & ((df_orders['order_datetime'].dt.day == 25) | (df_orders['order_datetime'].dt.day == 31))
    )
)

In [8]:
df_orders['hour_of_day'] = df_orders['order_datetime'].dt.hour
df_orders['day_of_week'] = df_orders['order_datetime'].dt.dayofweek

### Customer features

In [9]:
last_orders = df_orders.groupby('customer_id')['order_datetime'].max()
max_datetime = df_orders['order_datetime'].max()
max_datetime

Timestamp('2017-02-27 23:00:00')

In [10]:
# customer_features['last_order_age_days'] = 
last_order_age_days = (max_datetime - last_orders).dt.days

In [11]:
last_order_age_days.name = 'last_order_age_days'

In [12]:
customer_features = last_order_age_days.to_frame()

In [13]:
customer_features.head()

,last_order_age_days
customer_id,
000097eabfd9,618
0000e2c6d9be,395
000133bb597f,1
00018269939b,22
0001a00468a6,573


In [16]:
n_orders_holidays = df_orders.groupby('customer_id')['is_holiday'].sum()
n_orders_holidays.name = 'n_orders_holidays'

In [23]:
customer_features = customer_features.join(n_orders_holidays)

In [22]:
mean_hour_of_day = df_orders.groupby('customer_id')['hour_of_day'].mean()
mean_hour_of_day.name = 'mean_hour_of_day'

In [24]:
customer_features = customer_features.join(mean_hour_of_day)

In [25]:
std_hour_of_day = df_orders.groupby('customer_id')['hour_of_day'].std().fillna(0)
std_hour_of_day.name = 'std_hour_of_day'

In [26]:
customer_features = customer_features.join(std_hour_of_day)

In [27]:
mean_day_of_week = df_orders.groupby('customer_id')['day_of_week'].mean()
mean_day_of_week.name = 'mean_day_of_week'

In [28]:
customer_features = customer_features.join(mean_day_of_week)

In [29]:
std_day_of_week = df_orders.groupby('customer_id')['day_of_week'].std().fillna(0)
std_day_of_week.name = 'std_day_of_week'

In [30]:
customer_features = customer_features.join(std_day_of_week)

In [31]:
customer_features.head()

,last_order_age_days,n_orders_holidays,mean_hour_of_day,std_hour_of_day,mean_day_of_week,std_day_of_week
customer_id,,,,,,
000097eabfd9,618,0,19.0,0.0,5.0,0.0
0000e2c6d9be,395,0,20.0,0.0,4.0,0.0
000133bb597f,1,0,19.0,0.0,6.0,0.0
00018269939b,22,0,17.0,0.0,6.0,0.0
0001a00468a6,573,0,19.0,0.0,1.0,0.0


In [32]:
first_orders = df_orders.groupby('customer_id')['order_datetime'].min()

In [33]:
# customer_features['first_order_age_days'] = 
first_order_age_days = (max_datetime - first_orders).dt.days

In [34]:
first_order_age_days.name = 'first_order_age_days'

In [35]:
customer_features = customer_features.join(first_order_age_days)

In [36]:
number_of_orders_per_customer = df_orders.groupby('customer_id')['amount_paid'].count()
number_of_orders_per_customer.name = 'n_orders'

In [37]:
number_of_orders_per_customer

customer_id
000097eabfd9    1
0000e2c6d9be    1
000133bb597f    1
00018269939b    1
0001a00468a6    1
               ..
fffd696eaedd    1
fffe9d5a8d41    3
ffff347c3cfa    2
ffff4519b52d    1
ffffccbfc8a4    1
Name: n_orders, Length: 245453, dtype: int64

In [38]:
customer_features = customer_features.join(number_of_orders_per_customer)

In [39]:
max_customer_order_rank = df_orders.groupby(
    'customer_id'
)['customer_order_rank'].max()
max_customer_order_rank

customer_id
000097eabfd9    1
0000e2c6d9be    1
000133bb597f    1
00018269939b    1
0001a00468a6    1
               ..
fffd696eaedd    1
fffe9d5a8d41    1
ffff347c3cfa    2
ffff4519b52d    1
ffffccbfc8a4    1
Name: customer_order_rank, Length: 245453, dtype: int64

In [40]:
max_customer_order_rank.name = 'max_customer_order_rank'

In [41]:
customer_features = customer_features.join(max_customer_order_rank)

In [42]:
failed_orders_per_customer = df_orders.groupby('customer_id')['is_failed'].sum()
failed_orders_per_customer.name = 'n_failed'

In [43]:
customer_features = customer_features.join(failed_orders_per_customer)
customer_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245453 entries, 000097eabfd9 to ffffccbfc8a4
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   last_order_age_days      245453 non-null  int64  
 1   n_orders_holidays        245453 non-null  int64  
 2   mean_hour_of_day         245453 non-null  float64
 3   std_hour_of_day          245453 non-null  float64
 4   mean_day_of_week         245453 non-null  float64
 5   std_day_of_week          245453 non-null  float64
 6   first_order_age_days     245453 non-null  int64  
 7   n_orders                 245453 non-null  int64  
 8   max_customer_order_rank  245453 non-null  int64  
 9   n_failed                 245453 non-null  int64  
dtypes: float64(4), int64(6)
memory usage: 30.6+ MB


In [44]:
max_voucher_amount = df_orders.groupby('customer_id')['voucher_amount'].max()
max_voucher_amount.name = 'max_voucher_amount'

In [45]:
customer_features = customer_features.join(max_voucher_amount)
customer_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245453 entries, 000097eabfd9 to ffffccbfc8a4
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   last_order_age_days      245453 non-null  int64  
 1   n_orders_holidays        245453 non-null  int64  
 2   mean_hour_of_day         245453 non-null  float64
 3   std_hour_of_day          245453 non-null  float64
 4   mean_day_of_week         245453 non-null  float64
 5   std_day_of_week          245453 non-null  float64
 6   first_order_age_days     245453 non-null  int64  
 7   n_orders                 245453 non-null  int64  
 8   max_customer_order_rank  245453 non-null  int64  
 9   n_failed                 245453 non-null  int64  
 10  max_voucher_amount       245453 non-null  float64
dtypes: float64(5), int64(6)
memory usage: 32.5+ MB


In [46]:
tot_voucher_amount = df_orders.groupby('customer_id')['voucher_amount'].sum()
tot_voucher_amount.name = 'tot_voucher_amount'

In [47]:
customer_features = customer_features.join(tot_voucher_amount)
customer_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245453 entries, 000097eabfd9 to ffffccbfc8a4
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   last_order_age_days      245453 non-null  int64  
 1   n_orders_holidays        245453 non-null  int64  
 2   mean_hour_of_day         245453 non-null  float64
 3   std_hour_of_day          245453 non-null  float64
 4   mean_day_of_week         245453 non-null  float64
 5   std_day_of_week          245453 non-null  float64
 6   first_order_age_days     245453 non-null  int64  
 7   n_orders                 245453 non-null  int64  
 8   max_customer_order_rank  245453 non-null  int64  
 9   n_failed                 245453 non-null  int64  
 10  max_voucher_amount       245453 non-null  float64
 11  tot_voucher_amount       245453 non-null  float64
dtypes: float64(6), int64(6)
memory usage: 34.3+ MB


In [48]:
n_vouchers = (df_orders['voucher_amount']>0).groupby(df_orders['customer_id']).sum()
n_vouchers.name = 'n_vouchers'

In [49]:
customer_features = customer_features.join(n_vouchers)

In [50]:
tot_delivery_fee = df_orders.groupby('customer_id')['delivery_fee'].sum()
max_delivery_fee = df_orders.groupby('customer_id')['delivery_fee'].max()
# How many times a delivery fee was paid
n_delivery_fee = (df_orders['delivery_fee']>0).groupby(df_orders['customer_id']).sum()

In [51]:
tot_delivery_fee.name = 'tot_delivery_fee'
max_delivery_fee.name = 'max_delivery_fee'
n_delivery_fee.name = 'n_delivery_fee'

In [52]:
customer_features = customer_features.join(tot_delivery_fee)
customer_features = customer_features.join(max_delivery_fee)
customer_features = customer_features.join(n_delivery_fee)
customer_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245453 entries, 000097eabfd9 to ffffccbfc8a4
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   last_order_age_days      245453 non-null  int64  
 1   n_orders_holidays        245453 non-null  int64  
 2   mean_hour_of_day         245453 non-null  float64
 3   std_hour_of_day          245453 non-null  float64
 4   mean_day_of_week         245453 non-null  float64
 5   std_day_of_week          245453 non-null  float64
 6   first_order_age_days     245453 non-null  int64  
 7   n_orders                 245453 non-null  int64  
 8   max_customer_order_rank  245453 non-null  int64  
 9   n_failed                 245453 non-null  int64  
 10  max_voucher_amount       245453 non-null  float64
 11  tot_voucher_amount       245453 non-null  float64
 12  n_vouchers               245453 non-null  int64  
 13  tot_delivery_fee         245453 non-null  float

In [53]:
tot_amount_paid = df_orders.groupby('customer_id')['amount_paid'].sum()
avg_amount_paid = df_orders.groupby('customer_id')['amount_paid'].mean()
max_amount_paid = df_orders.groupby('customer_id')['amount_paid'].max()
min_amount_paid = df_orders.groupby('customer_id')['amount_paid'].min()

In [54]:
tot_amount_paid.name = 'tot_amount_paid'
avg_amount_paid.name = 'avg_amount_paid'
max_amount_paid.name = 'max_amount_paid'
min_amount_paid.name = 'min_amount_paid'

In [55]:
customer_features = customer_features.join(tot_amount_paid)
customer_features = customer_features.join(avg_amount_paid)
customer_features = customer_features.join(max_amount_paid)
customer_features = customer_features.join(min_amount_paid)

In [56]:
customer_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245453 entries, 000097eabfd9 to ffffccbfc8a4
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   last_order_age_days      245453 non-null  int64  
 1   n_orders_holidays        245453 non-null  int64  
 2   mean_hour_of_day         245453 non-null  float64
 3   std_hour_of_day          245453 non-null  float64
 4   mean_day_of_week         245453 non-null  float64
 5   std_day_of_week          245453 non-null  float64
 6   first_order_age_days     245453 non-null  int64  
 7   n_orders                 245453 non-null  int64  
 8   max_customer_order_rank  245453 non-null  int64  
 9   n_failed                 245453 non-null  int64  
 10  max_voucher_amount       245453 non-null  float64
 11  tot_voucher_amount       245453 non-null  float64
 12  n_vouchers               245453 non-null  int64  
 13  tot_delivery_fee         245453 non-null  float

In [57]:
n_restaurants = df_orders.groupby('customer_id')['restaurant_id'].nunique()
n_restaurants.name = 'n_restaurants'
n_restaurants

customer_id
000097eabfd9    1
0000e2c6d9be    1
000133bb597f    1
00018269939b    1
0001a00468a6    1
               ..
fffd696eaedd    1
fffe9d5a8d41    2
ffff347c3cfa    2
ffff4519b52d    1
ffffccbfc8a4    1
Name: n_restaurants, Length: 245453, dtype: int64

In [58]:
customer_features = customer_features.join(n_restaurants)
customer_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245453 entries, 000097eabfd9 to ffffccbfc8a4
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   last_order_age_days      245453 non-null  int64  
 1   n_orders_holidays        245453 non-null  int64  
 2   mean_hour_of_day         245453 non-null  float64
 3   std_hour_of_day          245453 non-null  float64
 4   mean_day_of_week         245453 non-null  float64
 5   std_day_of_week          245453 non-null  float64
 6   first_order_age_days     245453 non-null  int64  
 7   n_orders                 245453 non-null  int64  
 8   max_customer_order_rank  245453 non-null  int64  
 9   n_failed                 245453 non-null  int64  
 10  max_voucher_amount       245453 non-null  float64
 11  tot_voucher_amount       245453 non-null  float64
 12  n_vouchers               245453 non-null  int64  
 13  tot_delivery_fee         245453 non-null  float

In [59]:
n_cities = df_orders.groupby('customer_id')['city_id'].nunique()
n_cities.name = 'n_cities'
n_cities

customer_id
000097eabfd9    1
0000e2c6d9be    1
000133bb597f    1
00018269939b    1
0001a00468a6    1
               ..
fffd696eaedd    1
fffe9d5a8d41    1
ffff347c3cfa    1
ffff4519b52d    1
ffffccbfc8a4    1
Name: n_cities, Length: 245453, dtype: int64

In [60]:
customer_features = customer_features.join(n_cities)

In [61]:
customer_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245453 entries, 000097eabfd9 to ffffccbfc8a4
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   last_order_age_days      245453 non-null  int64  
 1   n_orders_holidays        245453 non-null  int64  
 2   mean_hour_of_day         245453 non-null  float64
 3   std_hour_of_day          245453 non-null  float64
 4   mean_day_of_week         245453 non-null  float64
 5   std_day_of_week          245453 non-null  float64
 6   first_order_age_days     245453 non-null  int64  
 7   n_orders                 245453 non-null  int64  
 8   max_customer_order_rank  245453 non-null  int64  
 9   n_failed                 245453 non-null  int64  
 10  max_voucher_amount       245453 non-null  float64
 11  tot_voucher_amount       245453 non-null  float64
 12  n_vouchers               245453 non-null  int64  
 13  tot_delivery_fee         245453 non-null  float

In [62]:
customer_features.head()

,last_order_age_days,n_orders_holidays,mean_hour_of_day,std_hour_of_day,mean_day_of_week,std_day_of_week,first_order_age_days,n_orders,max_customer_order_rank,n_failed,...,n_vouchers,tot_delivery_fee,max_delivery_fee,n_delivery_fee,tot_amount_paid,avg_amount_paid,max_amount_paid,min_amount_paid,n_restaurants,n_cities
customer_id,,,,,,,,,,,,,,,,,,,,,
000097eabfd9,618,0,19.0,0.0,5.0,0.0,618,1,1,0,...,0,0.000,0.000,0,11.46960,11.46960,11.46960,11.46960,1,1
0000e2c6d9be,395,0,20.0,0.0,4.0,0.0,395,1,1,0,...,0,0.000,0.000,0,9.55800,9.55800,9.55800,9.55800,1,1
000133bb597f,1,0,19.0,0.0,6.0,0.0,1,1,1,0,...,0,0.493,0.493,1,5.93658,5.93658,5.93658,5.93658,1,1
00018269939b,22,0,17.0,0.0,6.0,0.0,22,1,1,0,...,0,0.493,0.493,1,9.82350,9.82350,9.82350,9.82350,1,1
0001a00468a6,573,0,19.0,0.0,1.0,0.0,573,1,1,0,...,0,0.493,0.493,1,5.15070,5.15070,5.15070,5.15070,1,1


### Store to disk

In [87]:
customer_features.to_json(
    "../data/customer_features.json",
    orient='table',
)